In [37]:
import cv2
import numpy as np
from IPython.display import Image
from IPython import display
from matplotlib import pyplot
import time

In [38]:
# Подключение предобученной модели YOLO
net = cv2.dnn.readNet('yolo/yolov3.weights','yolo/yolov3.cfg') 
# Считывание классов
with open('yolo/coco.names.txt') as f:
    labels = f.read().strip().split('\n')
# Получение всех выходных слоев    
layer_names = net.getLayerNames()
out_layers_indexes_arr = net.getUnconnectedOutLayers()
out_layers_indexes = [index[0] - 1 for index in out_layers_indexes_arr]
out_layers_names = [layer_names[index] for index in out_layers_indexes ]
out_layers_names

['yolo_82', 'yolo_94', 'yolo_106']

In [39]:
# Прямоугольники, определяющие область с машинами на фото
boxes = [
    [30,400,500,740],
    [500,385,800,740],
    [800,430,1080,740]
]

In [59]:
def draw_boxes(img):
    color = [255,0,0]
    width = 3
    
    for x1, y1, x2, y2 in boxes:
        img = cv2.rectangle(img, (x1,y1),(x2,y2), color, width)
        
    return img

# Проверка на нахождение конктретных объектов в исследумой области
def check_coords(x,y,label):
    if label not in ['car','truck','bus','motorbike']:
        return False
    
    for x1, y1, x2, y2 in boxes:
        if x1 <= x <= x2 and y1 <= y <= y2:
            return True
    return False

# Выделение прямоугольником объектов
def draw_object(img, x,y,w,h, label, success):
    x1, y1 = x-w//2, y-h//2
    x2, y2 = x+w//2, y+h//2
    color = [0,255,0] if success else [0,0,255]
    width = 2
    
    img = cv2.rectangle(img, (x1,y1),(x2,y2), color, width)
    
    font_size = 1 
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = label
    
    img = cv2.putText(img, text, (x1,y1-10), font, font_size, color, width)
    
    return img

history = []

# Подсчет машин на парковке
def draw_cars_count(img, count):
    history.append([time.time(),count])
    timestamp = time.time() - 30
    filtered_history = []
    for t, c in history:
        if t >= timestamp:
            filtered_history.append(c)
            
    mid_count = int(sum(filtered_history)/len(filtered_history))
    
    color = [255,0,0]
    width = 5
    font_size = 3
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = str(mid_count) + ' cars'
    img = cv2.putText(img, text, (50,150), font, font_size, color, width)
    
    text = str(21-count) + ' free'
    img = cv2.putText(img, text, (50,250), font, font_size, color, width)
    
    return img

# Подача фото на вход CNN
def apply_yolo(img):
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255, (608,608),(0,0,0), swapRB=True)
    net.setInput(blob)
    out_layers = net.forward(out_layers_names)
    object_boxes = []
    object_probas = []
    object_labels = []

    for layer in out_layers:
        for result in layer:
            x, y, w, h = result[:4]
            x = int(x*width)
            w = int(w*width)
            y = int(y*height)
            h = int(h*height)
            probas = result[5:]
            max_proba_index = np.argmax(probas)
            max_proba = probas[max_proba_index]
            if max_proba > 0:
                object_boxes.append([x, y, w, h])
                object_probas.append(float(max_proba))
                object_labels.append(labels[max_proba_index])
                
    filtered_boxes_indexes = cv2.dnn.NMSBoxes(object_boxes, object_probas, 0, 0.27)
    
    img = draw_boxes(img)
    success_count = 0 
    
    for index_arr in filtered_boxes_indexes:
        index = index_arr[0]
        box = object_boxes[index]
        x, y, w, h = box
        success = check_coords(x, y, object_labels[index])
        img = draw_object(img, x, y, w, h, object_labels[index], success)
        if success:
            success_count += 1
            
    img = draw_cars_count(img, success_count)
    
    return img

In [1]:
# Считывание кадров со стрима
for i in range(10):
    cap = cv2.VideoCapture('http://96.66.39.30:8090/cgi-bin/faststream.jpg')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.rectangle(frame, (760,560),(1080,780), [0,0,0], thickness = -1)   
        frame = apply_yolo(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


        pyplot.figure(figsize=(20,15))
        pyplot.imshow(frame)
        display.clear_output(wait=True)
        display.display(pyplot.gcf())        

    cap.release()

NameError: name 'cv2' is not defined